## Colab Mount Drive


In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


## Set Main Pathes

In [1]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False



In [2]:
import platform

if platform.system() == "Linux":
    if is_colab():
      print("Running in Google Colab")
      path_main_folder = "/content/drive/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"


else:
    print("Running on Windows")
    path_main_folder = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"


path_data_folder = path_main_folder + "AIST_Cleaned/data2_mosheerIsmail/"#"AIST_Cleaned/data2_mosheerIsmail/"
path_cfg = path_data_folder +"cfg.sumocfg"
path_project_folder = path_main_folder + ""



Running on Windows


In [3]:
# prompt: check if file path ok

import os

if os.path.exists(path_main_folder):
  print(f"File exists at: {path_main_folder}")
else:
  print(f"File does not exist at: {path_main_folder}")


File exists at: I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/


In [4]:
yaml_file = path_project_folder + "config.yaml"
keys_file = path_project_folder + "keys.env"

##  install sumo

In [ ]:
!sudo add-apt-repository -y ppa:sumo/stable
!sudo apt-get update
!sudo apt-get install -y sumo sumo-tools sumo-doc


In [ ]:
!source ~/.bashrc


In [ ]:
!sumo --version


## Install Packages

In [ ]:
!pip install traci
!pip install dotenv
!pip install ray
!pip install ray[rllib]
!pip install colorama
!pip install optuna
!pip install stable_baselines3

## Import Packages

In [5]:
import sys
sys.path.append(path_project_folder)


In [6]:
from Connections import SumoConnection
from Connections.Connection import *
import os
import traci
import gymnasium as gym
from numpy import inf
import numpy as np
import SumoEnvSingleAgent
from Utils_reporting import *
from Utils_running_singleAgent import *
from rewards import *
from stable_baselines3 import PPO ,DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import torch
import time
from Callbacks import *
from models.d3qn import D3QNAgent
from Utils_reporting import *


## Set Parameters

In [7]:
max_step = 4000
precent_Scale = .38
is_gui =False

In [8]:
scale = int(10*precent_Scale)

In [9]:
agent_id='1698478721'

## Open Sumo

In [10]:
if platform.system() == "Linux":
    if is_colab():
        print("Running in Google Colab")
        sumo_home = str(os.getenv("sumo_home_lin"))
        sumo_binary = str(os.getenv("sumo_binary_lin"))
        if is_gui:
            print("GUI not supported on linux")
    else:
      print("Not running in HPC Linux (default linux)")


else:
    print("Running on Windows")
    print("Running in Google Colab")
    sumo_home = str(os.getenv("sumo_home_win"))
    if is_gui:
        sumo_binary = str(os.getenv("sumo_binary_gui_win"))
    else:
        sumo_binary = str(os.getenv("sumo_binary_cli_win"))




Running on Windows
Running in Google Colab


In [11]:
python_path =sumo_home
data_path = path_data_folder

In [12]:
cmd=[sumo_binary, "-c", path_cfg ,"--log", "sumo_log.txt", "--verbose", "true"]


In [13]:
conn=SumoConnection.SumoConnection(cmd)

No connection found, creating a new connection ❌


## Load Traffic Lights info

In [14]:
traffic_lights,policies=get_traffic_lights_policies_high_group(durations=[1],agent_ids=[agent_id])
traffic_lights

[(4, '1698478721')]

## Start Simulation

In [15]:
traci.simulation.setScale(scale)

In [16]:
time_before =time.time()

for i in range(max_step):
    conn.do_steps_duration(1,max_step,traffic_lights[0][1])

time_after =time.time()


## Results

In [17]:
time_diff = time_after - time_before
print(Fore.GREEN + f"Time taken for training: {round(time_diff,3)} seconds ({round(time_diff/60,3)} Minutes)" + Style.RESET_ALL)


Time taken for training: 19.636 seconds (0.327 Minutes)


In [18]:
def append_to_file(filename, text):
    with open(filename, 'a') as file:
        file.write(text + '\n')  # '\n' لتضيف السطر الجديد بعد كل جملة


In [19]:
traci.close()

In [20]:
results = conn.get_sumo_statics(python_path, data_path)
for key, value in results.items():
    print(Fore.CYAN + f"{key}: {round(value,3)}" + Style.RESET_ALL)
    #append_to_file("output1.txt",f"{key}: {round(value,3)}")


Successfully ran C:\Program Files (x86)\Eclipse\Sumo\tools\xml\xml2csv.py with I:\My Drive\study\graduation_project\final\Code\project_files\TrafficManager\TrafficManager\AIST_Cleaned\data2_mosheerIsmail\osm.statistics.xml
Waiting_Time: 137.24
Speed: 8.84
Waiting_Vehicles: 4568.0
TimeLoss: 147.73
Depart_Delay: 139.09
